In [1]:
from pathlib import Path
import sys
import time
import threading

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
%matplotlib inline
import autoclasswrapper as wrapper
import os 

afterall = {}
subfeatures = ['P15', 'P16', 'P17', 'P18', 'P19']
os.environ['PATH'] += ":/home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c"
#!touch /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
#!echo "xref_class_report_att_list = 0, 1 \nreport_mode = \"data\" \ncomment_data_headers_p = true " >> /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
for feature in subfeatures:
    runtimeDir = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round2/" + feature
    if not Path(runtimeDir).exists():
        os.mkdir(runtimeDir)
    os.chdir(runtimeDir)
    
            # Create object to prepare dataset.
    clust = wrapper.Input()

    # Load datasets from tsv files.
    pathData = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/" + feature + ".tsv"
    clust.add_input_data(pathData, "real location")
    clust.add_input_data("/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P21.tsv", "real location")
    
    # Prepare input data:
    # - create a final dataframe
    # - merge datasets if multiple inputs
    clust.prepare_input_data()

    # Create files needed by AutoClass.
    clust.create_db2_file()
    clust.create_hd2_file()
    clust.create_model_file()
    clust.create_sparams_file(max_duration = 0)
    clust.create_rparams_file()
    #!rm autoclass.r-params
    #!cp /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round1/autoclass.r-params .
    
    !rm -f autoclass-run-* *.results-bin


    # Search autoclass in path.
    wrapper.search_autoclass_in_path()

    # Create object to run AutoClass.
    run = wrapper.Run()

    # Prepare run script.
    run.create_run_file()

    # Run AutoClass.
    run.run()
        
    timer = 0
    step = 2
    while not Path(runtimeDir + "/autoclass-run-success").exists():
        timer += step
        sys.stdout.write("\r")
        sys.stdout.write(f"Time: {timer} sec.")
        sys.stdout.flush()
        time.sleep(step)
        if Path("autoclass-run-failure").exists():
            print("error")
            break
            
    if Path(runtimeDir + "/autoclass-run-success").exists():
        results = wrapper.Output()
        results.extract_results()
        results.aggregate_input_data()
#         results.write_cdt()
#         results.write_cdt(with_proba=True)
#         results.write_class_stats()
#         results.write_dendrogram()
        result = pd.read_csv(runtimeDir + '/autoclass_out.tsv', encoding="utf-8", header=0, sep='\t', index_col=0)
        label =  pd.read_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/label.tsv', encoding='utf-8', header=0, sep='\t', index_col=0)

        label['cluster'] = result['main-class']
        label.to_csv('app.csv', index = True, header=True)


        listH = []
        listApp = []
        listCluster = []
        def fun(x):
            listCluster.append(x['cluster'].values[0])
            listH.append(x['label'].value_counts().max()/x.shape[0])
            listApp.append(x['label'].value_counts().idxmax())


        label.groupby('cluster').apply(fun)

        H = sum(listH)/len(listH)
        print(listH)
        print(listApp)
        print(listCluster)
        print(H)

        dict = {'Cluster': listCluster, 'App': listApp, 'H': listH}

        saveResult = pd.DataFrame(dict)

        saveResult.to_csv(runtimeDir + '/resultCal.csv', index = True, header=True)
        
        afterall.update({feature:H})
        
round1 = pd.DataFrame(list(afterall.items()), index= range(len(subfeatures)))
round1.info()
round1.head(1)
round1.to_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round2/Hp4.csv', index = True, header=True)

2020-04-27 16:28:58 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P15.tsv' as 'real location' with error 0.01
2020-04-27 16:28:58 INFO     Detected encoding: ascii
2020-04-27 16:28:58 INFO     Found 3200 rows and 2 columns
2020-04-27 16:28:58 DEBUG    Checking column names
2020-04-27 16:28:58 DEBUG    Index name 'index'
2020-04-27 16:28:58 DEBUG    Column name 'P15'
2020-04-27 16:28:58 INFO     Checking data format
2020-04-27 16:28:58 INFO     Column 'P15'
2020-04-27 16:28:58 INFO     count    3200.000000
2020-04-27 16:28:58 INFO     mean        0.047417
2020-04-27 16:28:58 INFO     std         1.574035
2020-04-27 16:28:58 INFO     min       -14.499616
2020-04-27 16:28:58 INFO     50%        -0.111508
2020-04-27 16:28:58 INFO     max        33.695054
2020-04-27 16:28:58 INFO     ---
2020-04-27 16:28:58 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P21.tsv' as 'real location' with err

Time: 458 sec.

2020-04-27 16:36:37 INFO     Extracting autoclass results
2020-04-27 16:36:37 INFO     Found 3200 cases classified in 22 classes
2020-04-27 16:36:41 INFO     Aggregating input data
2020-04-27 16:36:41 INFO     Writing classes + probabilities .tsv file
2020-04-27 16:36:42 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P16.tsv' as 'real location' with error 0.01
2020-04-27 16:36:42 INFO     Detected encoding: ascii
2020-04-27 16:36:42 INFO     Found 3200 rows and 2 columns
2020-04-27 16:36:42 DEBUG    Checking column names
2020-04-27 16:36:42 DEBUG    Index name 'index'
2020-04-27 16:36:42 DEBUG    Column name 'P16'
2020-04-27 16:36:42 INFO     Checking data format
2020-04-27 16:36:42 INFO     Column 'P16'
2020-04-27 16:36:42 INFO     count    3200.000000
2020-04-27 16:36:42 INFO     mean       -0.006727
2020-04-27 16:36:42 INFO     std         1.063719
2020-04-27 16:36:42 INFO     min        -9.211523
2020-04-27 16:36:42 INFO     50%   

[0.0918580375782881, 0.06666666666666667, 0.09046454767726161, 0.0771513353115727, 0.22676579925650558, 0.14516129032258066, 0.1793103448275862, 0.09917355371900827, 1.0, 0.09259259259259259, 0.12264150943396226, 0.12195121951219512, 0.15492957746478872, 0.25, 0.16666666666666666, 0.11475409836065574, 0.96, 0.17073170731707318, 0.20930232558139536, 0.21428571428571427, 0.16666666666666666, 0.2]
['DROPBOX', 'APPLE', 'OFFICE_365', 'HTTP_PROXY', 'CONTENT_FLASH', 'DROPBOX', 'APPLE_ICLOUD', 'AMAZON', 'IP_ICMP', 'MS_ONE_DRIVE', 'INSTAGRAM', 'APPLE', 'APPLE_ICLOUD', 'DNS', 'WINDOWS_UPDATE', 'WINDOWS_UPDATE', 'DNS', 'EBAY', 'FACEBOOK', 'HTTP', 'REST_APP', 'WIKIPEDIA']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
0.2236851660564173


2020-04-27 16:36:42 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:36:42 INFO     Writing run file
2020-04-27 16:36:42 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:36:42 INFO     AutoClass C version: AUTOCLASS C (version 3.3.6unx)
2020-04-27 16:36:42 INFO     Running clustering...


...

[0.08823529411764706, 0.06451612903225806, 0.08430232558139535, 0.19946808510638298, 0.12867647058823528, 0.235, 0.12371134020618557, 0.18292682926829268, 0.14583333333333334, 0.14423076923076922, 0.10526315789473684, 1.0, 0.09523809523809523, 0.20689655172413793, 0.1232876712328767, 0.15151515151515152, 0.13157894736842105, 1.0, 0.3170731707317073, 0.23684210526315788, 0.21428571428571427, 0.125]
['APPLE_ICLOUD', 'HTTP', 'APPLE_ICLOUD', 'CONTENT_FLASH', 'OFFICE_365', 'DROPBOX', 'MS_ONE_DRIVE', 'SPOTIFY', 'APPLE', 'APPLE', 'HTTP_CONNECT', 'IP_ICMP', 'EBAY', 'DNS', 'DNS', 'GMAIL', 'REST_APP', 'DNS', 'HTTP_PROXY', 'SSL', 'HTTP', 'INSTAGRAM']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
0.23199459735084085


2020-04-27 17:03:06 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 17:03:06 INFO     Writing run file
2020-04-27 17:03:06 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 17:03:06 INFO     AutoClass C version: AUTOCLASS C (version 3.3.6unx)
2020-04-27 17:03:06 INFO     Running clustering...


Time: 460 sec.

2020-04-27 17:10:46 INFO     Extracting autoclass results
2020-04-27 17:10:46 INFO     Found 3200 cases classified in 28 classes
2020-04-27 17:10:49 INFO     Aggregating input data
2020-04-27 17:10:49 INFO     Writing classes + probabilities .tsv file


[0.07803468208092486, 0.07621951219512195, 0.08695652173913043, 0.18324607329842932, 0.08904109589041095, 0.15384615384615385, 0.09195402298850575, 0.08870967741935484, 0.2529411764705882, 0.10625, 0.09701492537313433, 0.4583333333333333, 0.19480519480519481, 0.08571428571428572, 0.08490566037735849, 1.0, 0.15384615384615385, 0.1511627906976744, 0.2926829268292683, 0.1780821917808219, 0.15151515151515152, 0.32, 1.0, 0.28, 0.15384615384615385, 0.23809523809523808, 0.12121212121212122, 0.2222222222222222]
['MS_ONE_DRIVE', 'HTTP_CONNECT', 'SSL', 'AMAZON', 'HTTP_PROXY', 'APPLE_ICLOUD', 'MS_ONE_DRIVE', 'INSTAGRAM', 'CONTENT_FLASH', 'APPLE_ITUNES', 'WINDOWS_UPDATE', 'DROPBOX', 'DNS', 'MSN', 'GMAIL', 'IP_ICMP', 'OFFICE_365', 'CLOUDFLARE', 'OFFICE_365', 'HTTP', 'DNS', 'APPLE', 'DNS', 'YAHOO', 'CONTENT_FLASH', 'HTTP_CONNECT', 'WIKIPEDIA', 'INSTAGRAM']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
0.22823704519916904
<class 'pandas.core.f